## test cells?


In [ ]:
const Mocha = require('mocha');

// call test runner
let mocha;
if (typeof mocha === 'undefined') {
    mocha = new Mocha({
        ui: 'bdd',
        reporter: 'list',
        reporterOptions: {
            stream: false,
        },
        //parallel: true,
        isWorker: true,
        jobs: 10,
        timeout: 10000,
    });
}

function resetTests(suite) {
    suite.tests.forEach(function (t) {
        delete t.state;
        t.timedOut = false;
    });
    suite.suites.forEach(resetTests);
    suite.tests = [];
}

function testCells(cells, ctx = {}) {
    resetTests(mocha.suite);
    if (!cells) {
        cells = 'mocha test the test cell runner';
    }
    if (!cells[0].code) {
        cells = typeof cells === 'string'
            && (!cells.includes('.ipynb') || cells.includes('['))
            ? [importer.interpret(cells)]
            : importer.interpret(cells);
    }
    cells.forEach(r => {
        mocha.suite.emit('pre-require', ctx, r.id, mocha);
        //Object.assign(ctx, Mocha);
        // TODO: add other require statements below to interent includes?
        try {
            console.log('testing cell:', r.id)
            var required = importer.import(r.id, ctx);
            Object.assign(ctx, required);
        } catch (e) {
            console.log(e);
        }
        mocha.suite.emit('require', required, r.id, mocha);
        mocha.suite.emit('post-require', ctx, r.id, mocha);
    });
    return new Promise(resolve => mocha.run(function (failures) {
        resolve(failures);
    }));
}

module.exports = testCells;



mocha test the test cell runner?


In [ ]:
const assert = require('assert');
const sinon = require('sinon');
const proxyquire = require('proxyquire');
const EventEmitter = require('events');
const Mocha = require('mocha')

describe('testCells', function () {
  let testCells;
  let importerStub;
  let mochaRunStub;
  let suiteMock;
  let mochaMock;

  beforeEach(() => {
    suiteMock = new EventEmitter();
    suiteMock.tests = [];
    suiteMock.suites = [];
    suiteMock.emit = sinon.spy();

    mochaRunStub = sinon.stub().callsFake(cb => cb(0));

    mochaMock = function () {
      return {
        suite: suiteMock,
        run: mochaRunStub
      };
    };
    mochaMock.prototype = function () {};
    mochaMock.prototype.ui = sinon.stub();
    mochaMock.prototype.reporter = sinon.stub();
    mochaMock.prototype.timeout = sinon.stub();

    importerStub = {
      interpret: sinon.stub(),
      import: sinon.stub()
    };

    testCells = importer.import('test cells', {
      require: function(string) {
        return proxyquire(string, {
          'mocha': mochaMock,
        })
      },
      importer: importerStub,
      cached: false,
     });
  });

  it('should handle null cells input and interpret string', async () => {
    importerStub.interpret.returns({ id: 'test-id', code: 'describe("sample", () => it("passes", () => {}));' });
    importerStub.import.returns({});

    const failures = await testCells(null);
    assert.strictEqual(failures, 0);
    sinon.assert.called(importerStub.interpret);
    sinon.assert.called(importerStub.import);
  });

  it('should interpret a string path (non-ipynb)', async () => {
    importerStub.interpret.returns([{ id: 'test-id', code: 'describe("test", function () {})' }]);
    importerStub.import.returns({});

    const failures = await testCells('test.js');
    assert.strictEqual(failures, 0);
  });

  it('should wrap raw code string as an array if not array', async () => {
    importerStub.interpret.returns({ id: 'inline-code', code: 'describe("inline", function () {})' });
    importerStub.import.returns({});

    const failures = await testCells('describe("inline", function () {})');
    assert.strictEqual(failures, 0);
  });

  it('should interpret array of cell objects', async () => {
    const cells = [{ id: 'a', code: 'describe("suite", function () {})' }];
    importerStub.import.returns({});

    const failures = await testCells(cells);
    assert.strictEqual(failures, 0);
  });

  it('should handle import error gracefully', async () => {
    const cells = [{ id: 'b', code: 'describe("suite", function () {})' }];
    importerStub.import.throws(new Error('import error'));

    const failures = await testCells(cells);
    assert.strictEqual(failures, 0);
  });
});


## test runner?

watch files run tests?

How to use: 

```
node -e "require('./Core').import('watch files run tests')('**/zuora*.ipynb', 'zuora to eloqua.ipynb')"

node -e "require('./Core').import('watch files run tests')('**/*.ipynb', 'test test runner')"```

TODO: restart this test script every loop? fork and abandon current thread?

TODO: update cell cache in intrepret notebooks.ipynb

TODO: git apply without whitespace, reset the rest?  separate index?  how do make git-scenario app?

TODO: re-import cells on Utilities/.modules folder change. run all "watcher" commands from a test below?  mocha.grep?





In [ ]:
var chokidar = require("chokidar");
var importer = require('../Core');
var testCells = importer.import('test cells');

// TODO: code analysis to combine blocks into modules?

var rateLimiter, done = true;
function testWatcher(files, tests) {
    files = typeof files === 'string' ? [files] : files;
    console.log('watching ' + files + ' - ' + path.resolve('.'))
    var watcher = chokidar.watch(files, {
        interval: 1000,
        atomic: 1000,
        awaitWriteFinish: true
    });
    watcher.on("change", function(event, path) {
        if(!done) {
            return;
        }
        console.log('running all tests');
        done = false;
        return testCells(tests).then(() => (done = true))
    });
    testCells(tests);
    
    var stdin = process.openStdin();
    stdin.addListener("data", function(d) {
        stdin.close();
    });
}
module.exports = testWatcher;
